In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from energizer.datastores import PandasDataStoreForSequenceClassification
from energizer.estimator import Estimator
from transformers import AutoModelForSequenceClassification
from typing import Dict, List
import torch
from torchmetrics import MetricCollection
from torchmetrics.classification import Accuracy, F1Score, Precision, Recall
from transformers import AutoModelForSequenceClassification
from energizer.enums import InputKeys, OutputKeys, RunningStage
import numpy as np
from energizer.utilities import move_to_cpu
from lightning.fabric.loggers import TensorBoardLogger
from lightning.fabric import seed_everything
from energizer.callbacks import GradNorm, PytorchTensorboardProfiler, EarlyStopping, ModelCheckpoint
from energizer.strategies import RandomStrategy, UncertaintyBasedStrategy

In [ ]:
ds = PandasDataStoreForSequenceClassification.load("./agnews_datastore/")

In [ ]:
class EstimatorForSequenceClassification(Estimator):

    def train_step(self, model, batch, batch_idx, loss_fn, metrics: MetricCollection) -> Dict:
        return self.step(model, batch, metrics, RunningStage.TRAIN)

    def validation_step(self, model, batch, batch_idx, loss_fn, metrics: MetricCollection) -> Dict:
        return self.step(model, batch, metrics, RunningStage.VALIDATION)

    def test_step(self, model, batch, batch_idx, loss_fn, metrics: MetricCollection) -> Dict:
        return self.step(model, batch, metrics, RunningStage.TEST)
    
    def train_epoch_end(self, output: List[np.ndarray], metrics: MetricCollection) -> Dict:
        return self.epoch_end(output, metrics, RunningStage.TRAIN)

    def validation_epoch_end(self, output: List[np.ndarray], metrics: MetricCollection) -> Dict:
        return self.epoch_end(output, metrics, RunningStage.VALIDATION)

    def test_epoch_end(self, output: List[np.ndarray], metrics: MetricCollection) -> Dict:
        return self.epoch_end(output, metrics, RunningStage.TEST)

    def step(self, model, batch: Dict, metrics: MetricCollection, stage: RunningStage) -> torch.Tensor:
        _ = batch.pop(InputKeys.ON_CPU, None)

        out = model(**batch)
        out_metrics = metrics(out.logits, batch[InputKeys.TARGET])

        if stage == RunningStage.TRAIN:
            logs = {OutputKeys.LOSS: out.loss, **out_metrics}
            self.log_dict({f"{stage}/{k}": v for k, v in logs.items()}, step=self.tracker.global_batch)

        return out.loss
    
    def epoch_end(self, output: List[np.ndarray], metrics: MetricCollection, stage: RunningStage) -> float:
        aggregated_metrics = move_to_cpu(metrics.compute())  # NOTE: metrics are still on device
        aggregated_loss = round(np.mean(output).item(), 6)
        
        logs = {OutputKeys.LOSS: aggregated_loss, **aggregated_metrics}
        self.log_dict({f"{stage}_end/{k}": v for k, v in logs.items()}, step=self.tracker.safe_global_epoch)

        return aggregated_metrics

    def configure_metrics(self, *_) -> MetricCollection:
        num_classes = self.model.num_labels
        task = "multiclass"
        # NOTE: you are in charge of moving it to the correct device
        return MetricCollection(
            {
                "accuracy": Accuracy(task, num_classes=num_classes),
                "f1_macro": F1Score(task, num_classes=num_classes, average="macro"),
                "precision_macro": Precision(task, num_classes=num_classes, average="macro"),
                "recall_macro": Recall(task, num_classes=num_classes, average="macro"),
                "f1_micro": F1Score(task, num_classes=num_classes, average="micro"),
                "precision_micro": Precision(task, num_classes=num_classes, average="micro"),
                "recall_micro": Recall(task, num_classes=num_classes, average="micro"),
            }
        ).to(self.device)

---
Random strategy

In [ ]:
class RandomStrategyForSequenceClassification(EstimatorForSequenceClassification, RandomStrategy):
    ...


seed_everything(42)
model = AutoModelForSequenceClassification.from_pretrained(
    ds.tokenizer.name_or_path,
    id2label=ds.id2label,
    label2id=ds.label2id,
    num_labels=len(ds.labels),
)

estimator = RandomStrategyForSequenceClassification(
    model, 
    accelerator="gpu", 
    loggers=[TensorBoardLogger("./", name="tb_logs")],
    callbacks=[
        GradNorm(2), 
        ModelCheckpoint("./checkpoints", monitor="f1_macro", stage="train", mode="max"),
        EarlyStopping(monitor="f1_macro", stage="train", interval="epoch", mode="max"),
    ],
    seed=42,
)

In [ ]:
ds.prepare_for_loading()
results = estimator.active_fit(
    datastore=ds, 
    query_size=50,
    max_rounds=20, 
    min_steps=50,
    reinit_model=False,
    # limit_pool_batches=10, 
    # limit_test_batches=10,
)

---
Entropy strategy

In [ ]:
class UncertaintyBasedStrategyForSequenceClassification(EstimatorForSequenceClassification, UncertaintyBasedStrategy):
    def pool_step( self, model, batch: Dict, batch_idx: int, metrics: MetricCollection) -> Dict:
        _ = batch.pop(InputKeys.ON_CPU)  # this is already handled in the `evaluation_step`
        logits = model(**batch).logits
        return self.score_fn(logits)

seed_everything(42)
model = AutoModelForSequenceClassification.from_pretrained(
    ds.tokenizer.name_or_path,
    id2label=ds.id2label,
    label2id=ds.label2id,
    num_labels=len(ds.labels),
)

estimator = UncertaintyBasedStrategyForSequenceClassification(
    score_fn="entropy",
    model=model, 
    accelerator="gpu", 
    loggers=[TensorBoardLogger("./", name="tb_logs")],
    callbacks=[
        GradNorm(2), 
        ModelCheckpoint("./checkpoints", monitor="f1_macro", stage="train", mode="max"),
        EarlyStopping(monitor="f1_macro", stage="train", interval="epoch", mode="max"),
    ],
)

In [ ]:
ds.prepare_for_loading()
results = estimator.active_fit(
    datastore=ds, 
    query_size=50,
    max_rounds=20, 
    min_steps=50,
)